In [ ]:
# Third-party
import astropy.coordinates as coord
from astropy.coordinates import SkyCoord
import astropy.units as u
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

# Custom
import gala.coordinates as gc
import gala.dynamics as gd
from gala.dynamics.mockstream import fardal_stream, streakline_stream
import gala.integrate as gi
import gala.potential as gp
from gala.units import galactic
from matplotlib import cm
from scipy.optimize import minimize

from potential import default_mw
from coordinates import galcen_frame, pal5_c

from density import DensityModel2D, get_projected_coords
from likelihood import a_to_z, Model

Make a mock stream model for testing:

In [ ]:
w0 = gd.PhaseSpacePosition(pal5_c.transform_to(galcen_frame).cartesian)
orbit = default_mw.integrate_orbit(w0, dt=-0.5, n_steps=8000, 
                                   Integrator=gi.DOPRI853Integrator)

stream = gd.mockstream.fardal_stream(default_mw, orbit[::-1], 
                                     5E4*u.Msun, release_every=1)
_ = stream.plot(marker='.', alpha=0.1)

We're going to do density fitting in Pal5 coordinates:

In [ ]:
sim_c = stream.to_coord_frame(gc.Pal5, galactocentric_frame=galcen_frame)

# Only grab data within bounds:
lon_limits = [-50, 50]*u.deg
lat_limits=[-10, 40]*u.deg

mask = ((sim_c.phi1.wrap_at(180*u.deg) > lon_limits[0]) &
        (sim_c.phi1.wrap_at(180*u.deg) < lon_limits[1]) & 
        (sim_c.phi2 > lat_limits[0]) &
        (sim_c.phi2 < lat_limits[1]))
sim_c = sim_c[mask]

In [ ]:
X = get_projected_coords(sim_c, pal5_c).T
mask = (X[:, 0] > -1) & (X[:, 1] > -1)
X = X[mask]

In [ ]:
spacing = 1.5 # MAGIC NUMBER
dens_model = DensityModel2D(X, poly_deg=5)
track = dens_model.get_dense_poly_track(size=10000)
nodes = dens_model.set_nodes(track=track, spacing=spacing)
# nodes = dens_model.set_nodes(track=track, nodes=nodes[1:-1])

In [ ]:
plt.figure(figsize=(10, 10))
plt.scatter(nodes[:, 0], nodes[:, 1])
plt.plot(dens_model.X[:, 0], dens_model.X[:, 1], 
         marker='.', ls='none', alpha=1, zorder=-10, color='k')
plt.xlim(-5, 40)

In [ ]:
bins = (np.linspace(-5, 45, 250),
        np.linspace(-5, 45, 250))
bincs = [0.5*(x[1:]+x[:-1]) for x in bins]
xg, yg = np.meshgrid(*bincs)
X_grid = np.stack((xg.ravel(), yg.ravel())).T

## Optimize:

In [ ]:
h = 0.5 * spacing
model = Model(dens_model, h=h, l=2.5)

In [ ]:
K = dens_model.K
print(K)

In [ ]:
sk0 = 0.5 * np.ones(K)

a0 = np.ones(K)/K
z0 = a_to_z(a0)

m0 = np.zeros(K)

In [ ]:
p0 = {'ln_s': np.log(sk0),
      'ln_z': np.log(z0),
      'm': m0}
x0 = model.pack_pars(**p0)
model.ln_posterior(x0)

In [ ]:
ln_dens = model.ln_density(p0, X_grid)

fig, ax = plt.subplots(1, 1, figsize=(6, 6))
ax.pcolormesh(xg, yg, np.exp(ln_dens.reshape(xg.shape) - ln_dens.max()),
              cmap='Blues')
ax.set_aspect('equal')

In [ ]:
def likelihood_helper(x):
    return -model.ln_posterior(x)

def deriv_helper(x):
    p = model.unpack_pars(x)
    return -model.derivs(p)

In [ ]:
bounds = [[-8, 4]]*K + [[-8, -1e-9]]*(K-1) + [[-5, 5]]*K

res = minimize(likelihood_helper, jac=deriv_helper, 
               x0=x0, method='L-BFGS-B', 
               bounds=bounds,
               options=dict(ftol=1e-13))
res

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 5.5), 
                         sharex=True, sharey=True)

ax = axes[0]
# ax.plot(dens_model.proj_xy[0], dens_model.proj_xy[1], 
#         marker='.', ls='none', alpha=0.4, zorder=-10, color='k')
H,xe,ye = np.histogram2d(dens_model.X[:,0], dens_model.X[:,1],
                         bins=bins)
ax.pcolormesh(xe, ye, H.T, cmap='Blues')

ax = axes[1]
ln_dens = model.ln_density(p0, X_grid)
ax.pcolormesh(xg, yg, np.exp(ln_dens.reshape(xg.shape) - ln_dens.max()),
              cmap='Blues')

ax = axes[2]
best_p = model.unpack_pars(res.x)
ln_dens = model.ln_density(best_p, X_grid)
H2 = np.exp(ln_dens.reshape(xg.shape) - ln_dens.max())
ax.pcolormesh(xg, yg, H2,
              cmap='Blues')

for ax in axes:
    ax.set_aspect('equal')

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 10))
ax.pcolormesh(xg, yg, H.T,
              cmap='Blues')
ax.set_aspect('equal')
ax.set_title('data')
fig.savefig('../plots/stream-data.png', dpi=250)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 10))
ax.pcolormesh(xg, yg, H2,
              cmap='Blues')
ax.set_aspect('equal')
ax.set_title('model')
fig.savefig('../plots/stream-model.png', dpi=250)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 10))
ax.pcolormesh(xg, yg, H2,
              cmap='Blues', 
              norm=mpl.colors.LogNorm(vmin=1e-5, vmax=1e0))
ax.plot(X[:, 0], X[:, 1], 
        marker='.', ls='none', alpha=0.5, zorder=10, color='k')
ax.set_aspect('equal')
ax.set_xlim(bincs[0].min(), bincs[0].max())
ax.set_ylim(bincs[1].min(), bincs[1].max())
fig.savefig('../plots/stream-compare.png', dpi=250)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 10))
ax.pcolormesh(xg, yg, H2,
              cmap='Blues', vmin=0, vmax=1e-2)
ax.set_aspect('equal')
ax.xaxis.set_visible(False)
ax.yaxis.set_visible(False)

In [ ]:
plt.scatter(dens_model.nodes[:, 0],
            dens_model.nodes[:, 1])

mu = model.get_mu(best_p)
plt.scatter(mu[:, 0], mu[:, 1])

In [ ]:
# plt.plot(dens_model.nodes[:, 0],
#          dens_model.nodes[:, 1] - mu[:, 1])

In [ ]:
plt.plot(nodes[:, 0], ak)